In [1]:
import numpy as np
import pandas as pd
import joblib

*** DATA CLEANING ***

In [3]:
# TODO: convert this all to functions.
# Load data
df = pd.read_excel("raw data/esmeralda.xlsx", engine="openpyxl")
pd.set_option("display.max_columns", None)
df.head()

,Fecha,Even_mon,ID,Especie,Escenario,Estrategia,Tipo,No_Nuc/Faj/Mod,Línea,No_Ind,Sup_Ind,Est_Fit,COL,VB,HER,PAT,COM,DM,ANE,PNE,Alt_Ind,DAB1_Ind,DAB2_Ind,DAB_Prom,Diam1_Ind,Diam2_Ind,Area_cop_Ind,Observaciones
0,2020-11-01,T1,1,Verbesina crassiramea,Pastos limpios,Nucleación,1.0,1,1.0,1,SI,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,9,NaN,9,30,32,480.0,NaN
1,2020-11-01,T1,2,Verbesina crassiramea,Pastos limpios,Nucleación,1.0,1,1.0,2,SI,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,NaN
2,2020-11-01,T1,3,Verbesina crassiramea,Pastos limpios,Nucleación,1.0,1,1.0,3,SI,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78,19,NaN,19,94,62,2914.0,NaN
3,2020-11-01,T1,4,Lupinus interruptus,Pastos limpios,Nucleación,1.0,1,1.0,4,SI,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89,10,NaN,10,63,38,1197.0,NaN
4,2020-11-01,T1,5,Lupinus interruptus,Pastos limpios,Nucleación,1.0,1,1.0,5,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,Muerto


In [34]:
df.shape

(11943, 28)

In [4]:
# Start converting to the proper datatypes
df = df.convert_dtypes()
df = df._convert(numeric=True)

str_cols = ['Even_mon', 'Especie', 'Escenario', 'Estrategia', 'Observaciones']
int_cols = ['ID', 'Tipo', 'No_Nuc/Faj/Mod', 'Línea', 'No_Ind', 'Est_Fit']
float_cols = ['Alt_Ind', 'DAB1_Ind', 'DAB2_Ind', 'DAB_Prom', 'Diam1_Ind', 'Diam2_Ind', 'Area_cop_Ind']
bool_cols = ['Sup_Ind', 'COL', 'VB', 'HER', 'PAT', 'COM', 'DM', 'ANE', 'PNE']

# Convert to ML-ready vectors
change = {'SI': 1, 'NO': 0}
for col in bool_cols:
    df[col] = df[col].map(change)
    df[col].fillna(0, inplace=True)
    
# Est_Fit is a categorical column, so get dummies.
# by ignoring the NA values we can get k-1 categorical columns
dummy_cols = pd.get_dummies(df['Est_Fit'], prefix='est', dtype='int')
df = pd.concat([df, dummy_cols], join='outer', axis=1)

# Fill last NA values
# General health level is na if the plant is dead. We can replace this with a value of 5
df['Est_Fit'].fillna(5, inplace=True)

# Correct clerical errors.
df.Especie = df.Especie.str.capitalize()
df.Especie.replace(to_replace='Solanum oblongifolium ', value='Solanum oblongifolium', inplace=True)
df.Escenario.replace(to_replace='Veg. Secundaria', value='Veg. secundaria', inplace=True)

In [5]:
# Save data
df.to_pickle('clean_data.p')